# **Downloads and Imports**

In [1]:
!pip install --quiet requests openai jsonschema pydantic[email]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 8.9 MB/s eta 0:00:00


In [4]:
import json
from openai import OpenAI
from google.colab import userdata
from typing import List, Dict, Literal

# **Configurations and Model Instances**

In [5]:
CHAT_MODEL = "llama-3.3-70b-versatile"
STRUCTURED_OUTPUT_MODEL = "openai/gpt-oss-20b"

In [6]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key = userdata.get("GROQ_API_KEY")
)

# **Task 1: Managing Conversation History with Summarization**

- Message History Tracking
- Summarization
- Truncation

In [7]:
MESSAGE_HISTORY : List[dict] = []
SUMMARY = ""

## Function to add messages

In [18]:
# summarization_interval,  The interval / turn count after which history summarization will happen
turn_count = 0  # Variable to track turn
def add_message(message:dict, summarization_interval:int = 8 ):
    global turn_count
    # Add message to history and increment turn
    MESSAGE_HISTORY.append(message)
    turn_count += 1

    print(f"Turn {turn_count}: Added {message}")

    # Check if periodic summarization is needed
    if turn_count % summarization_interval == 0:
      print(f"Turn count {turn_count}, output: {turn_count % summarization_interval}")
      periodic_summarization()

## functionm to create summary

In [21]:
def generate_summary(messages):

    if not MESSAGE_HISTORY:
        print("No messages to summarize.")
        return

    # Convert messsage history which is in dictionary format, to a string
    conversation_text = "\n".join([f"{message['role']}: {message['content']}" for message in MESSAGE_HISTORY])

    summary_prompt = f"""
        Please provide a concise summary of the following conversation, in the third person point of view. Focus on:
        1. Main topics discussed
        2. Key decisions or outcomes
        3. Important information exchanged
        4. Overall context and flow

        Conversation:
        {conversation_text}

        Summary:"""
    try:
        chat_bot_response = client.chat.completions.create(
          model=CHAT_MODEL,
            messages=[{"role":"user", "content":conversation_text}],
            max_tokens=150,
            temperature=0.3)

        summary = chat_bot_response.choices[0].message.content.strip()

        print(f"✅ Generated summary ({len(summary)} characters)")
        return summary
    except Exception as e:
        print(f"❌ Error generating summary: {e}")
        return "Summary generation failed"

## Function to create periodic summary

In [22]:
def periodic_summarization():
        global MESSAGE_HISTORY
        global SUMMARY

        # Only generate a new summary if there's enough new conversation
        if len(MESSAGE_HISTORY) >= 2:
            current_summary = generate_summary(MESSAGE_HISTORY)

            if SUMMARY:
                # Combine with previous summary
                combined_prompt = f"""
                Previous summary: {SUMMARY}

                New conversation summary: {current_summary}

                Please create a comprehensive summary that combines both:"""

                try:
                    response = client.chat.completions.create(
                        model=CHAT_MODEL,
                        messages=[{"role": "user", "content": combined_prompt}],
                        max_tokens= 300,
                        temperature=0.3
                    )

                    SUMMARY = response.choices[0].message.content.strip()

                except Exception as e:
                    print(f"❌ Error combining summaries: {e}")
                    SUMMARY = current_summary
            else:
                SUMMARY = current_summary

            # Replace history with summary
            MESSAGE_HISTORY = [
                {"role": "system", "content": f"Previous conversation summary: {SUMMARY}"}
            ]

            print(f"Conversation summarized and history reset")
            print(f"Current summary length: {len(SUMMARY)} characters")

## Helper Functions

In [11]:

def count_words(text: str) -> int:
    return len(text.split())

def count_characters(messages: List[Dict]) -> int:
    return sum(len(msg["content"]) for msg in messages)

def count_total_words(messages: List[Dict]) -> int:
    return sum(count_words(msg["content"]) for msg in messages)

## Function to truncate conversation by turns

In [33]:
# max_turns, number of messages after which we want to truncate
def truncate_by_turns(messages: List[Dict], max_turns:int) -> List[Dict]:
  if max_turns and len(messages) > max_turns:
      truncated = messages[-max_turns:]
      print(f"Truncated to last {max_turns} messages")
      return truncated
  return messages

## Truncate by length

In [16]:
# max_characters
# max_words
def truncate_by_length(messages: List[Dict], max_characters:int, max_words:int) -> List[Dict]:
    if not (max_characters or max_words):
        return messages

    result = []
    current_chars = 0
    current_words = 0

    # Start from the end to keep most recent messages
    for message in reversed(messages):
        msg_chars = len(message["content"])
        msg_words = count_words(message["content"])

        # Check if adding this message would exceed limits
        if (max_characters and current_chars + msg_chars > max_characters) or (max_words and current_words + msg_words > max_words):
            break

        result.insert(0, message)
        current_chars += msg_chars
        current_words += msg_words

    if len(result) < len(messages):
        removed = len(messages) - len(result)
        print(f"📝 Truncated {removed} messages due to length constraints")

    return result

## **Test Conversation Manager**

## Try Conversation with LLM, test summarization

In [24]:
# Start chat with LLM
turn_count = 0 # reset turn count
while True:
    user_input = input("User: ")
    add_message({"role": "user", "content": user_input})

    try:
        chat_bot_response = client.chat.completions.create(
            model=CHAT_MODEL,
            messages=MESSAGE_HISTORY
        )
        response_content = chat_bot_response.choices[0].message.content
        add_message({"role": "assistant", "content": response_content})

        print(f"Bot: {response_content}")

    except Exception as e:
        print(f"An error occurred: {e}")

User: Hello
Turn 1: Added {'role': 'user', 'content': 'Hello'}
Turn 2: Added {'role': 'assistant', 'content': 'Hello. Is there something I can help you with or would you like to chat?'}
Bot: Hello. Is there something I can help you with or would you like to chat?
User: What is a context window,  tell me in one words
Turn 3: Added {'role': 'user', 'content': 'What is a context window,  tell me in one words'}
Turn 4: Added {'role': 'assistant', 'content': 'Memory.'}
Bot: Memory.
User: Can a prompt be considered in  context window?
Turn 5: Added {'role': 'user', 'content': 'Can a prompt be considered in  context window?'}
Turn 6: Added {'role': 'assistant', 'content': 'Yes.'}
Bot: Yes.
User: Nice, on how many GPUs you weas trained?
Turn 7: Added {'role': 'user', 'content': 'Nice, on how many GPUs you weas trained?'}
Turn 8: Added {'role': 'assistant', 'content': '256-512 GPUs.'}
Turn count 8, output: 0
✅ Generated summary (474 characters)
Conversation summarized and history reset
Current 

KeyboardInterrupt: Interrupted by user

In [25]:
SUMMARY

"Here's a comprehensive summary that combines both:\n\nI'm a large language model with a maximum token capacity of approximately 2048 tokens and a context window of around 2048 tokens. My training process involved a massive corpus of text and a distributed computing architecture that utilized a cluster of 256-512 GPUs, as well as hundreds of GPUs in a broader sense. This robust infrastructure enabled me to learn from a vast amount of text data. Previously, we discussed my capabilities and training process, and now I'm ready to help with any new questions or topics you'd like to discuss. To recap, my context window allows me to consider up to 2048 tokens of previous conversation history, including prompts, conversation history, and any other relevant text, when generating a response. Feel free to ask me anything, and I'll do my best to provide helpful and accurate information. Whether you have follow-up questions about my capabilities or want to explore new subjects, I'm here to assist 

## Test truncation with test message jsons

In [31]:
conversation_samples = [
    ("user", "Hi! I need advice on starting a food truck business."),
    ("assistant", "That's exciting! What type of cuisine are you considering for your food truck?"),
    ("user", "I'm interested in serving gourmet tacos and fresh juices."),
    ("assistant", "Great choice! Have you picked a location or checked local food truck regulations?"),
    ("user", "Not yet. Do you have tips on selecting a good spot?"),
    ("assistant", "Absolutely. Look for areas with heavy foot traffic, local events, and nearby office parks."),
    ("user", "Thanks! What licenses do I need to get started?"),
    ("assistant", "You'll need a mobile food vendor license, health permits, and possibly parking permits depending on your city."),
    ("user", "Got it. What about marketing ideas for my launch?"),
    ("assistant", "Consider social media campaigns, local food festivals, and offering promo deals for first-time customers."),
    ("user", "That's helpful. How much investment do I need to get started?"),
    ("assistant", "Typically, starting a food truck costs between $50,000 and $150,000, depending on equipment, customization, and permits.")
]


In [36]:
# Clear previous history for a clean test
MESSAGE_HISTORY = []
turn_count = 0
SUMMARY = ""

print("-------- Testing Truncation --------")

# fill message history with sample conversation
for role, content in conversation_samples:
    add_message({"role": role, "content": content})

print("\n-------- Original Message History --------")
for msg in MESSAGE_HISTORY:
    print(f"{msg['role']}: {msg['content']}")
print(f"Total messages: {len(MESSAGE_HISTORY)}")

print("\n-------- Truncating by Turns (max_turns=3) --------")
truncated_by_turns = truncate_by_turns(MESSAGE_HISTORY, max_turns=3)
for msg in truncated_by_turns:
    print(f"{msg['role']}: {msg['content']}")
print(f"Truncated messages: {len(truncated_by_turns)}")


print("\n-------- Truncating by Length (max_characters=300, max_words=80) --------")
truncated_by_length = truncate_by_length(MESSAGE_HISTORY, max_characters=300, max_words=80)
for msg in truncated_by_length:
    print(f"{msg['role']}: {msg['content']}")
print(f"Truncated messages: {len(truncated_by_length)}")

print("\n-------- Truncation Test Complete --------")

-------- Testing Truncation --------
Turn 1: Added {'role': 'user', 'content': 'Hi! I need advice on starting a food truck business.'}
Turn 2: Added {'role': 'assistant', 'content': "That's exciting! What type of cuisine are you considering for your food truck?"}
Turn 3: Added {'role': 'user', 'content': "I'm interested in serving gourmet tacos and fresh juices."}
Turn 4: Added {'role': 'assistant', 'content': 'Great choice! Have you picked a location or checked local food truck regulations?'}
Turn 5: Added {'role': 'user', 'content': 'Not yet. Do you have tips on selecting a good spot?'}
Turn 6: Added {'role': 'assistant', 'content': 'Absolutely. Look for areas with heavy foot traffic, local events, and nearby office parks.'}
Turn 7: Added {'role': 'user', 'content': 'Thanks! What licenses do I need to get started?'}
Turn 8: Added {'role': 'assistant', 'content': "You'll need a mobile food vendor license, health permits, and possibly parking permits depending on your city."}
Turn coun

# **Task 2: JSON Schema Classification & Information Extraction**

## Required JSON Schema and Prompts

In [43]:
from pydantic import BaseModel, EmailStr
from typing import List, Dict, Literal, Optional

class UserInformationModel(BaseModel):
    name: Optional[str] = None
    email: Optional[EmailStr] = None
    phone: Optional[str] = None
    location: Optional[str] = None
    age: Optional[int] = None
    tech_stack: Optional[List[str]] = None

In [44]:
SYSTEM_PROMPT = system_prompt = """You are an expert information extractor. Your task is to carefully analyze chat conversations and extract specific user information when available.


FOLLOW THESE IMPORTANT GUIDELINES:
1. Extract only information that is explicitly mentioned in the conversation
2. Do not make assumptions or infer information not directly stated
3. For names: Extract full names when provided
4. For emails: Look for email addresses in standard formats, should contain '@'
5. For phones: Extract any phone number mentioned (any format, should be 10 digit)
6. For locations: Extract cities, states, addresses, or geographical references
7. For age: Extract numerical age when mentioned
8. For tech_stack: Extract a list of technical skills when mentioned

Cross check all information before giving an output.

If information is not clearly given, leave that field empty instead of guessing."""

## Function to extract information

In [45]:
def extract_information(chat_conversation: str | List[Dict]) -> dict:

  USER_PROMPT = f"""Please extract any available user information from this chat conversation:

{chat_conversation}

Use the extract_user_information function to provide the structured output."""
  try:
      chat_bot_response = client.chat.completions.create(
            model=STRUCTURED_OUTPUT_MODEL,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role":"user", "content":USER_PROMPT},
            ],
            response_format =  {
                "type": "json_schema",
                "json_schema":{
                    "name":"user_information",
                    "description":"User information extracted from the chat conversation",
                    "schema":UserInformationModel.model_json_schema()

                } }
        )
      return chat_bot_response.choices[0].message.content
  except Exception as e:
    raise Exception(f"Error extracting information: {e}")

## Test Information Extraction

In [46]:
SAMPLE_CHATS = [
    # Sample 1: Customer support chat with complete information, including tech stack
    """Customer: Hi, I'm having trouble with my order
Agent: I'd be happy to help! Can I get your name and email?
Customer: Sure, it's Sarah Johnson and my email is sarah.johnson@email.com
Agent: Thank you Sarah. Can I also get your phone number and address?
Customer: Yes, my phone is (555) 123-4567 and I live in Seattle, WA. I'm 28 years old by the way, does that matter for shipping?
Agent: Thank you for all that information! Just to confirm, what tech do you use the most?
Customer: Mostly Python and Django for my small business website.
Agent: Let me look up your order.""",

    # Sample 2: Registration chat with partial information & tech stack
    """User: I want to sign up for your newsletter
Bot: Great! What's your email address?
User: It's mike.chen@company.com
Bot: Thanks! And your name?
User: Mike Chen
Bot: Any location preference for local events?
User: I'm based in San Francisco.
Bot: Do you use any specific tech tools or frameworks at work?
User: Node.js and React.
Bot: Perfect, you're all set!""",

    # Sample 3: Survey chat with mixed information
    """Interviewer: Thank you for participating in our survey. Could you share some basic information?
Participant: Of course
Interviewer: What's your name and age?
Participant: I'm Jennifer Davis, 35 years old
Interviewer: Great! Do you have a preferred contact method?
Participant: You can reach me at jennifer.davis.work@gmail.com or call me at 555-987-6543
Interviewer: And which city are you located in?
Participant: I live in Austin, Texas
Interviewer: Do you work in any particular technology area?
Participant: Yes, mainly with AWS, Java, and Docker.
Interviewer: Perfect, that's all we need!""",

    # Sample 4: Job application chat with multiple tech skills
    """Recruiter: Welcome, could you please tell me your name and how to reach you?
Candidate: My name is Priya Sharma. You can email me at priya.sharma123@gmail.com.
Recruiter: And your phone number and current location?
Candidate: Sure, my phone is +91-9876543210, and I'm currently in Bengaluru.
Recruiter: What's your age and primary tech stacks?
Candidate: I'm 25. I primarily work with Vue.js, FastAPI, and PostgreSQL.
Recruiter: Noted. Do you have experience with cloud platforms?
Candidate: Yes, GCP and Azure are part of my daily workflow.""",

    # Sample 5: Casual onboarding chat with partial information
    """Mentor: Hi! Welcome aboard. What's your full name?
Newcomer: I'm Alex Martinez.
Mentor: Great, Alex. You can share your contact details when you're ready.
Newcomer: Sure! My email: alexmartinez@startup.io
Mentor: Where are you joining from?
Newcomer: I'm based in Boston. I usually build with Flutter and Firebase.
Mentor: Awesome! Age is optional, but it helps with mentorship pairing.
Newcomer: I'm 30.
Mentor: Thanks, Alex. Let's start!"""
]

In [48]:
print("--- Testing Information Extraction with Sample Chats ---")

for i, chat in enumerate(SAMPLE_CHATS):
    print(f"\n--- Sample Chat {i+1} ---")
    print(chat)
    try:
        extracted_data_json = extract_information(chat)

        extracted_data = json.loads(extracted_data_json)
        validated_data = UserInformationModel.model_validate(extracted_data)
        print("\n✅ Extracted and Validated Information:")
        print(validated_data.model_dump_json(indent=2))
    except Exception as e:
        print(f"\n❌ Error processing sample chat {i+1}: {e}")

print("\n--- Information Extraction Test Complete ---")

--- Testing Information Extraction with Sample Chats ---

--- Sample Chat 1 ---
Customer: Hi, I'm having trouble with my order
Agent: I'd be happy to help! Can I get your name and email?
Customer: Sure, it's Sarah Johnson and my email is sarah.johnson@email.com
Agent: Thank you Sarah. Can I also get your phone number and address?
Customer: Yes, my phone is (555) 123-4567 and I live in Seattle, WA. I'm 28 years old by the way, does that matter for shipping?
Agent: Thank you for all that information! Just to confirm, what tech do you use the most?
Customer: Mostly Python and Django for my small business website.
Agent: Let me look up your order.

✅ Extracted and Validated Information:
{
  "name": "Sarah Johnson",
  "email": "sarah.johnson@email.com",
  "phone": "(555) 123-4567",
  "location": "Seattle, WA",
  "age": 28,
  "tech_stack": [
    "Python",
    "Django"
  ]
}

--- Sample Chat 2 ---
User: I want to sign up for your newsletter
Bot: Great! What's your email address?
User: It's mi